In [1]:
from datetime import datetime
import json
import urllib
import urllib2
import re

from bs4 import BeautifulSoup

Get the `movie_id` from IMDB and create the reviews-URL. The next step is to load the page, get the contents and soup it.

In [2]:
movie_id = 'tt1596363'

url = 'http://www.imdb.com/title/' + movie_id + '/reviews'

In [3]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

def getUrlSoup(url,hdr):
    req1 = urllib2.Request(url, headers=hdr)
    page = urllib2.urlopen(req1)
    content = page.read()
    soup = BeautifulSoup(content)
    return soup
    
soup = getUrlSoup(url,hdr)

/Users/luuk/pyEnv/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


Obtain the main block and find out how many pages there are.

In [4]:
main_block = (soup.body.find('div', attrs={'class':'reviews'})
                       .find('div', attrs={'id':'tn15main'})
                       .find('div', attrs={'id':'tn15content'}))

In [5]:
num_pages = int(main_block.findAll('table')[1]
                          .find('td',attrs={'nowrap':'1'}).get_text().split(' of ')[1].rstrip(':'))

print 'There are {} pages to get the reviews from'.format(num_pages)

There are 21 pages to get the reviews from


Iterate over the pages and grab the reviews. We'll store all of them in the `reviews` variable.

In [6]:
def getReviews(main_block, reviews_obj=None):
    
    assert reviews_obj is not None
    
    # Treat it as a massive block of text and split it on the horizontal lines.
    reviews_block = re.split(r"<hr[^>]*>", str(main_block))
    
    # Iterate over all the reviews.
    # NOTE: The indexing is dubious here...
    for r in reviews_block[1:-3]:
        
        # Soup the block of html-code-text
        soup_review = BeautifulSoup(r)
        soup_review_header = soup_review.find('div')
        
        # Create an empty dictionary to hold the review.
        review = dict()
        
        # Get the user-name, user-link for the reviewer.
        review['user_name'] = soup_review_header.findAll('a')[1].get_text()
        review['user_href'] = soup_review_header.findAll('a')[1]['href']

        # Try to get the rating.
        soup_imgs = soup_review_header.findAll('img')
        review['review_rating'] = None
        if len(soup_imgs) > 1:
            review['review_rating'] = int(soup_imgs[1]['alt'].split('/')[0])
        
        # Get the review title.
        review['review_title'] = soup_review.find('h2').get_text()
        
        # Get the review content.
        tmp_content = soup_review.findAll('p')
        if len(tmp_content) > 1 and 'spoilers' in tmp_content[0].get_text():
            review['review_content'] = tmp_content[1].get_text()
        else:
            review['review_content'] = tmp_content[0].get_text()
        
        # Get the review usefulness.
        # review['review_useful'] = soup_review_header.find('small').get_text().split('people')[0].rstrip().lstrip()
        
        reviews_obj.append(review)
        
    # Done. Return the reviews_obj
    return reviews_obj

In [7]:
print 'Starting to grab the reviews for this movie...'

# Create an empty array to hold all the reviews.
REVIEWS = []

print '\nGetting the contents for page',
for i in range(1,num_pages+1):
    
    # If this is the first page, we don't need to load the new page.
    if i == 1:
        REVIEWS = getReviews(main_block,REVIEWS)
    
    # We need to grab the contents for the next page.
    else:
        page_soup = getUrlSoup(url + '?start=' + str(10*(i-1)), hdr)
        main_block = (page_soup.body.find('div', attrs={'class':'reviews'})
                                    .find('div', attrs={'id':'tn15main'})
                                    .find('div', attrs={'id':'tn15content'}))
        REVIEWS = getReviews(main_block,REVIEWS)
    
    print i, '.',


print '\n\nObtained {} reviews for this movie'.format(len(REVIEWS))

Starting to grab the reviews for this movie...

Getting the contents for page 1 . 2 . 3 . 4 . 5 . 6 . 7 . 8 . 9 . 10 . 11 . 12 . 13 . 14 . 15 . 16 . 17 . 18 . 19 . 20 . 21 . 

Obtained 203 reviews for this movie


In [10]:
print json.dumps(REVIEWS[97:99],indent=4)

[
    {
        "review_content": "\nOK, so i've watched this in movie hoping to understand the main story\nof '08 crisis. Instead I get a 2h movie full of boring terms and people\nacting like they are smart and \"gtfo attitude\". I've tried so hard to\nmaintain the course of my brain ,to pay attention to the story, but the\nreally annoying terminology and economical jargon kept me away from\nconstructing any bond with an actor or a situation. I am thinking: what\nif a broker watch this movie...what he will think about it ? It will be\ninteresting or probably will say that the movie it's not accurate like\npretty much every movie lately. Seriously, this production don't even\nput Chris bale in a good light... because he is just a \"mad pawn \"\nright there talking about banks and sums. His role don't put him in the\nright place for showing his potential. Crude !\n", 
        "review_rating": 5, 
        "user_href": "/user/ur29880112/", 
        "review_title": "Boring Apocalypse", 
  